<a href="https://colab.research.google.com/github/BinaryNavigator07/Network-Intrusion-Detection-System/blob/main/CICIDS%202017/2017_LSTM_without_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seaborn catboost --quiet
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
np.random.seed(178)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")
print("Dataset folder:", path)

# Find all CSV files in the dataset directory
csv_file_names = [f for f in os.listdir(path) if f.endswith(".csv")]
print(f"Found {len(csv_file_names)} CSV files.")

dfs = []
for file_name in csv_file_names:
    file_path = os.path.join(path, file_name)
    print("Loading:", file_path)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine all files into one DataFrame
df = pd.concat(dfs, ignore_index=True)
print("Shape of combined data:", df.shape)

# Preview
df.head()

In [ ]:
df

In [ ]:
df[' Label'].isnull().sum()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
from itertools import combinations

def data_cleaning(df):
    df.columns=df.columns.str.strip()
    print("Dataset Shape: ",df.shape)

    num=df._get_numeric_data()
    num[num<0]=0

    zero_variance_cols=[]
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(columns=zero_variance_cols,axis=1,inplace=True)
    print("Zero Variance Columns: ",zero_variance_cols, " are dropped!!")
    print("Shape after removing the zero variance columns: ",df.shape)

    df.replace([np.inf,-np.inf],np.nan,inplace=True)
    print(df.isna().any(axis=1).sum(), "rows dropped")
    df.dropna(inplace=True)
    print("Shape after Removing NaN: ",df.shape)

    df.drop_duplicates(inplace=True)
    print("Shape after dropping duplicates: ",df.shape)

    column_pairs = [(i,j) for i,j in combinations(df,2) if df[i].equals(df[j])]
    ide_cols=[]
    for col_pair in column_pairs:
        ide_cols.append(col_pair[1])
    df.drop(columns=ide_cols,axis=1,inplace=True)
    print("Columns which have identical values: ",column_pairs," dropped!")
    print("Shape after removing identical value columns: ",df.shape)
    return df
df=data_cleaning(df)

In [ ]:
df.columns

In [ ]:
df['Label'].value_counts()

In [ ]:
def change_label(df):
  df.Label.replace(['DoS Hulk','DDoS','DoS GoldenEye','DoS slowloris','DoS Slowhttptest','Heartbleed'],'Dos',inplace=True)
  df.Label.replace(['Web Attack � Brute Force','Web Attack � XSS','Web Attack � Sql Injection'],'WebAttack',inplace=True)
  df.Label.replace(['FTP-Patator','SSH-Patator'],'BruteForce',inplace=True)

In [ ]:
change_label(df)

In [ ]:
df['Label'].value_counts()

In [ ]:
Label = pd.DataFrame(df.Label)
Label

In [ ]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

In [ ]:
df

In [ ]:
X = df.drop('Label', axis=1)
y = df['Label']

In [ ]:
# One Hot encoding  (BENIGN, Bot, BruteForce, DoS, Infiltration, PortScan, WebAttack)

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

In [ ]:
X = X.reset_index(drop=True)

In [ ]:
X.info()

In [ ]:
df = pd.concat([X, y_encoded], axis=1)
df

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

In [ ]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

In [ ]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y=np.array(y)

In [ ]:
X_train = np.reshape(X_train, ( X_train.shape[0], 1 , X_train.shape[1] ))
X_test = np.reshape(X_test, ( X_test.shape[0], 1,  X_test.shape[1] ))

In [ ]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

In [ ]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# summary of model layers
model.summary()

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

In [ ]:
import tensorflow as tf
import subprocess

class GpuUsageLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n--- Epoch {epoch+1} GPU Usage ---")
        print(subprocess.getoutput("nvidia-smi --query-gpu=memory.used,memory.total,utilization.gpu --format=csv"))

with tf.device('/GPU:0'):
    history = model.fit(
        X_train, y_train,
        epochs=100,  # use small number for test
        batch_size=5000,
        validation_split=0.2,
        callbacks=[GpuUsageLogger()]
    )


In [ ]:
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

In [ ]:
model.save('LSTM_without_feature_extraction.keras')